<a href="https://colab.research.google.com/github/tctamedie/EventManagement/blob/master/FraudDetectionCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>